In [1]:
pip install youtube-comment-downloader

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Scroll down

In [ ]:
##ChromeDriver 설정
#options = Options()
##options.add_argument('--headless')  # GUI 없이 실행
#options.add_argument('--no-sandbox')
#options.add_argument('--disable-dev-shm-usage')

## WebDriver 생성
#driver = webdriver.Chrome(options=options)

## 유튜브 댓글 크롤링 함수
#def crawl_youtube_comments(driver, 방송, 식당):
#    search_query = f"{식당} {방송}"
#    driver.get(f"https://www.youtube.com/results?search_query={search_query}")
#    time.sleep(3)  # Wait for the search results page to load

#    # Collect video URLs
#    videos = driver.find_elements(By.CSS_SELECTOR, '#dismissible > ytd-thumbnail')[:5]
#    urls = []
#    for video in videos:
#        try:
#            video_url = video.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
#            if video_url:
#                urls.append(video_url)
#                print(f"수집된 URL: {video_url}")  # 디버깅용 출력
#        except Exception as e:
#            print(f"URL 수집 실패: {e}")

#    results = []
#    for index, url in enumerate(urls):
#        try:
#            driver.get(url)
#            # Wait for the page title to load
#            WebDriverWait(driver, 20).until(
#                EC.presence_of_element_located((By.CSS_SELECTOR, '#title > h1 > yt-formatted-string'))
#            )
#            video_title = driver.find_element(By.CSS_SELECTOR, '#title > h1 > yt-formatted-string').text
#            print(f"[{index + 1}] Title: {video_title}")

#            # Scroll to load comments
#            scroll_pause_time = 5
#            last_height = driver.execute_script("return document.documentElement.scrollHeight")
#            while True:
#                driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
#                time.sleep(scroll_pause_time)
#                new_height = driver.execute_script("return document.documentElement.scrollHeight")
#                if new_height == last_height:  # No more scrolling possible
#                    break
#                last_height = new_height

#            # Collect comments and users
#            try:
#                comment_box = driver.find_element(By.CSS_SELECTOR, '#sections')
#                comments = comment_box.find_elements(By.CSS_SELECTOR, '#content-text')
#                users = comment_box.find_elements(By.CSS_SELECTOR, 'a#author-text')
                
#                for comment, user in zip(comments, users):
#                    results.append({
#                        "식당명": f"{식당}",
#                        "방송": f"{방송}",
#                        "Title": video_title,
#                        "User": user.text.strip(),
#                        "Comment": comment.text.strip(),
#                    })
                
#            except Exception as e:
#                print(f"댓글 수집 실패: {e}")
            
#        except Exception as e:
#            print(f"Error processing video at {url}: {e}")
#            continue  # Skip to the next video

#    # Save results
#    if results:
#        try:
#            df = pd.DataFrame(results)
#            output_file = f"youtubecomments_{식당}.csv"
#            df.to_csv(output_file, index=False, encoding='utf-8-sig')
#            print(f"Saved comments to {output_file}")
#        except Exception as e:
#            print(f"Failed to save comments for {식당}: {e}")
#    else:
#        print(f"No comments collected for {식당}.")

##식당 정보 불러오기
#restaurants = pd.read_csv("restaurants_final.csv", index_col=False)
#restaurants = restaurants[['식당명', '방송']]
#restaurants = restaurants[41:]
#restaurants.sort_values(by='식당명', inplace=True)

## 모든 식당에 대해 크롤링 수행
#for _, row in restaurants.iterrows():
#    crawl_youtube_comments(driver, row['방송'], row['식당명'])

## 드라이버 종료
#driver.quit()

# URL

In [ ]:
# 식당 csv 파일 불러오기
restaurants = pd.read_csv("restaurants_final.csv", index_col=False)
restaurants = restaurants[['식당명', '방송']]
restaurants = restaurants[41:]
restaurants.sort_values(by='식당명', inplace=True)
restaurants['식당명'].unique()

In [ ]:
# ChromeDriver 설정
options = Options()
# options.add_argument('--headless')  # GUI 없이 실행 (필요 시 활성화)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# WebDriver 생성
driver = webdriver.Chrome(options=options)

# 유튜브 댓글 크롤링 함수
def crawl_youtube_comments(driver, url):
    results = []
    try:
        driver.get(url)
        # 동영상 제목 로드 대기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#title > h1 > yt-formatted-string'))
        )
        video_title = driver.find_element(By.CSS_SELECTOR, '#title > h1 > yt-formatted-string').text
        print(f"Processing Title: {video_title}")

        # 댓글 로드: 페이지 스크롤
        body = driver.find_element(By.TAG_NAME, 'body')
        for _ in range(15):  # 스크롤 횟수 조정 가능
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)

        # 댓글과 사용자 정보 수집
        comments = driver.find_elements(By.CSS_SELECTOR, '#content-text')
        users = driver.find_elements(By.CSS_SELECTOR, 'a#author-text')
        for comment, user in zip(comments, users):
            results.append({
                "URL": url,
                "Title": video_title,
                "User": user.text.strip(),
                "Comment": comment.text.strip(),
            })
    except Exception as e:
        print(f"Error processing video at {url}: {e}")
    return results

# URL 리스트
urls = [
    'https://www.youtube.com/watch?v=KcchqqFvFVo&pp=ygUc7Y-s67Cp7KO86r6466-4IOqzqOuqqeyLneuLuQ%3D%3D'
    'https://www.youtube.com/watch?v=rZRKzRc_UEo&pp=ygUc7Y-s67Cp7KO86r6466-4IOqzqOuqqeyLneuLuQ%3D%3D',
    'https://www.youtube.com/watch?v=7-Is-FkhPgo&pp=ygUc7Y-s67Cp7KO86r6466-4IOqzqOuqqeyLneuLuQ%3D%3D',
    'https://www.youtube.com/watch?v=VANU9Funja8&pp=ygUc7Y-s67Cp7KO86r6466-4IOqzqOuqqeyLneuLuQ%3D%3D',
    'https://www.youtube.com/watch?v=OWnurZ2-mT0&pp=ygUc7Y-s67Cp7KO86r6466-4IOqzqOuqqeyLneuLuQ%3D%3D',
]

# 모든 URL의 결과를 저장할 리스트
all_results = []

# 각 URL에 대해 크롤링 수행
for url in urls:
    results = crawl_youtube_comments(driver, url)
    all_results.extend(results)

# 모든 데이터를 하나의 CSV 파일로 저장
if all_results:
    output_file = "youtube_comments_포방주꾸미.csv"
    df = pd.DataFrame(all_results)
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"Saved all comments to {output_file}")

# 드라이버 종료
driver.quit()

In [ ]:
# Directory containing the CSV files
directory = os.getcwd()

# List to hold DataFrames
dataframes = []

# Iterate through all files in the directory
for file in os.listdir(directory):
    # Check if the file matches the pattern 'youtubecomments_{식당}.csv'
    if file.startswith("youtubecomments_") and file.endswith(".csv"):
        file_path = os.path.join(directory, file)
        try:
            # Read the CSV file and append to the list
            df = pd.read_csv(file_path)
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
output_file = os.path.join(directory, "combined_youtubecomments.csv")
combined_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"Combined CSV saved as {output_file}")